<a href="https://colab.research.google.com/github/axiomhaidarcreate/project-AI-/blob/main/Copy_of_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1️⃣ تثبيت المكتبات
!pip install ultralytics opencv-python deep-sort-realtime matplotlib scipy

# 2️⃣ استيراد المكتبات
from ultralytics import YOLO
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from google.colab.patches import cv2_imshow
import os
from scipy.ndimage import gaussian_filter

# 3️⃣ رفع الفيديوهات
uploaded = files.upload()  # اختر الفيديوهات من جهازك
videos = list(uploaded.keys())
print("Uploaded videos:", videos)

# 4️⃣ تحميل YOLOv8n والمتتبع
model = YOLO("yolov8n.pt")
tracker = DeepSort(max_age=30)

# اقتصار المعالجة على وسط الفيديو
roi_ratio = (0.2, 0.8, 0.2, 0.8)

# 5️⃣ معالجة كل فيديو
for video_path in videos:
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    output_name = f"output_{video_path}"
    out = cv2.VideoWriter(output_name, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))

    frame_id = 0
    people_count_per_frame = []  # لتخزين عدد الأشخاص في كل إطار
    heatmap_accum = np.zeros((height, width), dtype=np.float32)  # Heatmap

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_id += 1
            if frame_id % 2 != 0:  # معالجة كل إطار ثاني لتسريع
                continue

            # اقتصار على ROI
            h, w, _ = frame.shape
            y_start = int(h * roi_ratio[0])
            y_end = int(h * roi_ratio[1])
            x_start = int(w * roi_ratio[2])
            x_end = int(w * roi_ratio[3])
            roi = frame[y_start:y_end, x_start:x_end]

            # كشف الأشخاص
            results = model(roi)[0]
            detections = []
            for box in results.boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())
                # إضافة إزاحة ROI
                x1 += x_start
                x2 += x_start
                y1 += y_start
                y2 += y_start
                detections.append(([x1, y1, x2-x1, y2-y1], conf, cls))

            # تتبع الأشخاص
            tracks = tracker.update_tracks(detections, frame=frame)

            current_ids = []
            for track in tracks:
                if not track.is_confirmed():
                    continue
                x1, y1, x2, y2 = track.to_ltrb()
                track_id = track.track_id
                current_ids.append(track_id)

                # رسم المستطيل حول الشخص
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
                cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1)-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

                # تحديث Heatmap
                cx = int((x1 + x2)/2)
                cy = int((y1 + y2)/2)
                cx = max(0, min(cx, width - 1))
                cy = max(0, min(cy, height - 1))
                heatmap_accum[cy, cx] += 5  # زيادة التأثير للقوة العالية

            # حفظ عدد الأشخاص في الإطار الحالي
            people_count_per_frame.append(len(current_ids))

            # عرض عدد الأشخاص مباشرة على الفيديو
            cv2.putText(frame, f"People: {len(current_ids)}", (20,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)

            # حفظ الإطار في الفيديو الناتج
            out.write(frame)

            # عرض الفيديو أثناء المعالجة في Colab
            cv2_imshow(frame)

    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print(f"{video_path} processing done! Saved as {output_name}")

    # تحسين Heatmap مع Gaussian smoothing
    heatmap_accum = gaussian_filter(heatmap_accum, sigma=10)

    # رسم Heatmap النهائي
    plt.figure(figsize=(12,8))
    plt.imshow(heatmap_accum, cmap='inferno', interpolation='bilinear')
    plt.title(f"Heatmap - {video_path}", fontsize=16)
    plt.axis('off')
    plt.show()
    plt.imsave(f"heatmap_{video_path}.png", heatmap_accum, cmap='inferno')

    # رسم مخطط عدد الأشخاص لكل إطار
    plt.figure(figsize=(12,5))
    plt.plot(people_count_per_frame, color='blue')
    plt.title(f"عدد الأشخاص في كل إطار - {video_path}", fontsize=14)
    plt.xlabel("Frame")
    plt.ylabel("عدد الأشخاص")
    plt.show()

# تحميل الفيديوهات الناتجة وHeatmaps
for video_path in videos:
    files.download(f"output_{video_path}")
    files.download(f"heatmap_{video_path}.png")


In [ ]:
# 1️⃣ تثبيت المكتبات
!pip install ultralytics opencv-python deep-sort-realtime matplotlib

# 2️⃣ استيراد المكتبات
from ultralytics import YOLO
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from google.colab.patches import cv2_imshow
import os

# 3️⃣ رفع الفيديوهات
uploaded = files.upload()  # اختر الفيديوهات من جهازك
videos = list(uploaded.keys())
print("Uploaded videos:", videos)

# 4️⃣ تحميل YOLOv8n والمتتبع
model = YOLO("yolov8n.pt")
tracker = DeepSort(max_age=30)

# اقتصار المعالجة على وسط الفيديو
roi_ratio = (0.2, 0.8, 0.2, 0.8)

# 5️⃣ إعداد العنصر المرجعي للقياس
# على سبيل المثال: عنصر معروف الطول (باب أو شخص) في الفيديو
reference_height_cm = 200   # طول العنصر بالسم
reference_height_px = 220   # ارتفاع العنصر بالبكسل في الفيديو
scale = reference_height_cm / reference_height_px  # سم لكل بكسل

# 6️⃣ معالجة كل فيديو
for video_path in videos:
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    output_name = f"output_{video_path}"
    out = cv2.VideoWriter(output_name, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))

    frame_id = 0
    people_count_per_frame = []  # لتخزين عدد الأشخاص في كل إطار

    # قاموس لتخزين الأشخاص الذين تم رسمهم مسبقًا
    drawn_ids = {}

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_id += 1
            if frame_id % 2 != 0:  # معالجة كل إطار ثاني لتسريع
                continue

            # اقتصار على ROI
            h, w, _ = frame.shape
            y_start = int(h * roi_ratio[0])
            y_end = int(h * roi_ratio[1])
            x_start = int(w * roi_ratio[2])
            x_end = int(w * roi_ratio[3])
            roi = frame[y_start:y_end, x_start:x_end]

            # كشف الأشخاص
            results = model(roi)[0]
            detections = []
            for box in results.boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())
                # إضافة إزاحة ROI
                x1 += x_start
                x2 += x_start
                y1 += y_start
                y2 += y_start
                detections.append(([x1, y1, x2-x1, y2-y1], conf, cls))

            # تتبع الأشخاص
            tracks = tracker.update_tracks(detections, frame=frame)

            current_ids = []

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                current_ids.append(track_id)

                # رسم المستطيل فقط إذا لم يتم رسم الشخص مسبقًا
                if track_id not in drawn_ids:
                    x1, y1, x2, y2 = track.to_ltrb()
                    drawn_ids[track_id] = (x1, y1, x2, y2)

                    # رسم المستطيل حول الشخص
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)

                    # حساب الطول الحقيقي للشخص
                    box_height = y2 - y1
                    person_height_cm = box_height * scale

                    # عرض ID والطول فوق المستطيل
                    cv2.putText(frame, f"ID: {track_id} Height: {int(person_height_cm)} cm",
                                (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

            # حفظ عدد الأشخاص الحالي
            people_count_per_frame.append(len(current_ids))

            # عرض عدد الأشخاص مباشرة على الفيديو
            cv2.putText(frame, f"People: {len(current_ids)}", (20,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)

            # حفظ الإطار في الفيديو الناتج
            out.write(frame)

            # عرض الفيديو أثناء المعالجة في Colab
            cv2_imshow(frame)

    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print(f"{video_path} processing done! Saved as {output_name}")

    # رسم مخطط عدد الأشخاص لكل إطار
    plt.figure(figsize=(12,5))
    plt.plot(people_count_per_frame, color='blue')
    plt.title(f"عدد الأشخاص في كل إطار - {video_path}", fontsize=14)
    plt.xlabel("Frame")
    plt.ylabel("عدد الأشخاص")
    plt.show()

# تحميل الفيديوهات الناتجة
for video_path in videos:
    files.download(f"output_{video_path}")


ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
# 1️⃣ تثبيت المكتبات
!pip install ultralytics opencv-python deep-sort-realtime matplotlib

# 2️⃣ استيراد المكتبات
from ultralytics import YOLO
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from google.colab.patches import cv2_imshow
import os

# 3️⃣ رفع الفيديوهات
uploaded = files.upload()  # اختر الفيديوهات من جهازك
videos = list(uploaded.keys())
print("Uploaded videos:", videos)

# 4️⃣ تحميل YOLOv8n والمتتبع
model = YOLO("yolov8n.pt")
# تحسين تتبع الأشخاص ليحتفظ بالـ ID حتى لو اختفى مؤقتاً
tracker = DeepSort(max_age=60, n_init=3)

# اقتصار المعالجة على وسط الفيديو (يمكن توسيع ROI حسب الحاجة)
roi_ratio = (0.2, 0.8, 0.2, 0.8)

# إعداد العنصر المرجعي للقياس (مثال: باب أو عنصر معروف الطول)
reference_height_cm = 200   # طول العنصر بالسم
reference_height_px = 220   # ارتفاع العنصر بالبكسل
scale = reference_height_cm / reference_height_px  # سم لكل بكسل

# 5️⃣ معالجة كل فيديو
for video_path in videos:
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    output_name = f"output_{video_path}"
    out = cv2.VideoWriter(output_name, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))

    frame_id = 0
    people_count_per_frame = []  # لتخزين عدد الأشخاص في كل إطار

    # قاموس لتخزين موقع كل شخص بشكل ديناميكي لتحديث المستطيل
    tracked_objects = {}

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_id += 1

            # اقتصار على ROI
            h, w, _ = frame.shape
            y_start = int(h * roi_ratio[0])
            y_end = int(h * roi_ratio[1])
            x_start = int(w * roi_ratio[2])
            x_end = int(w * roi_ratio[3])
            roi = frame[y_start:y_end, x_start:x_end]

            # كشف الأشخاص
            results = model(roi)[0]
            detections = []
            for box in results.boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())
                # تصفية فقط الأشخاص المؤكدين
                if cls == 0 and conf > 0.5:
                    # إضافة إزاحة ROI
                    x1 += x_start
                    x2 += x_start
                    y1 += y_start
                    y2 += y_start
                    detections.append(([x1, y1, x2-x1, y2-y1], conf, cls))

            # تتبع الأشخاص
            tracks = tracker.update_tracks(detections, frame=frame)

            current_ids = []

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                current_ids.append(track_id)

                # تحديث موقع الشخص في tracked_objects
                x1, y1, x2, y2 = track.to_ltrb()
                tracked_objects[track_id] = (x1, y1, x2, y2)

                # رسم المستطيل ومؤشر الطول في كل إطار
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)

                # حساب الطول الحقيقي للشخص
                box_height = y2 - y1
                person_height_cm = box_height * scale

                # عرض ID والطول فوق المستطيل
                cv2.putText(frame, f"ID: {track_id} Height: {int(person_height_cm)} cm",
                            (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

            # حفظ عدد الأشخاص الحالي
            people_count_per_frame.append(len(current_ids))

            # عرض عدد الأشخاص مباشرة على الفيديو
            cv2.putText(frame, f"People: {len(current_ids)}", (20,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)

            # حفظ الإطار في الفيديو الناتج
            out.write(frame)

            # عرض الفيديو أثناء المعالجة في Colab
            cv2_imshow(frame)

    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print(f"{video_path} processing done! Saved as {output_name}")

    # رسم مخطط عدد الأشخاص لكل إطار
    plt.figure(figsize=(12,5))
    plt.plot(people_count_per_frame, color='blue')
    plt.title(f"عدد الأشخاص في كل إطار - {video_path}", fontsize=14)
    plt.xlabel("Frame")
    plt.ylabel("عدد الأشخاص")
    plt.show()

# تحميل الفيديوهات الناتجة
for video_path in videos:
    files.download(f"output_{video_path}")


In [ ]:
# 1️⃣ تثبيت المكتبات
!pip install ultralytics opencv-python deep-sort-realtime matplotlib

# 2️⃣ استيراد المكتبات
from ultralytics import YOLO
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from google.colab.patches import cv2_imshow
import os

# 3️⃣ رفع الفيديوهات
uploaded = files.upload()  # اختر الفيديوهات من جهازك
videos = list(uploaded.keys())
print("Uploaded videos:", videos)

# 4️⃣ تحميل YOLOv8n والمتتبع
model = YOLO("yolov8n.pt")
# تحسين تتبع الأشخاص ليحتفظ بالـ ID حتى لو اختفى مؤقتاً
tracker = DeepSort(max_age=60, n_init=3)

# اقتصار المعالجة على وسط الفيديو (يمكن توسيع ROI حسب الحاجة)
roi_ratio = (0.2, 0.8, 0.2, 0.8)

# إعداد العنصر المرجعي للقياس (مثال: باب أو عنصر معروف الطول)
reference_height_cm = 200   # طول العنصر بالسم
reference_height_px = 220   # ارتفاع العنصر بالبكسل
scale = reference_height_cm / reference_height_px  # سم لكل بكسل

# 5️⃣ معالجة كل فيديو
for video_path in videos:
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    output_name = f"output_{video_path}"
    out = cv2.VideoWriter(output_name, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))

    frame_id = 0
    people_count_per_frame = []  # لتخزين عدد الأشخاص في كل إطار

    # قاموس لتخزين موقع كل شخص بشكل ديناميكي لتحديث المستطيل
    tracked_objects = {}

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_id += 1

            # اقتصار على ROI
            h, w, _ = frame.shape
            y_start = int(h * roi_ratio[0])
            y_end = int(h * roi_ratio[1])
            x_start = int(w * roi_ratio[2])
            x_end = int(w * roi_ratio[3])
            roi = frame[y_start:y_end, x_start:x_end]

            # كشف الأشخاص
            results = model(roi)[0]
            detections = []
            for box in results.boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())
                # تصفية فقط الأشخاص المؤكدين
                if cls == 0 and conf > 0.5:
                    # إضافة إزاحة ROI
                    x1 += x_start
                    x2 += x_start
                    y1 += y_start
                    y2 += y_start
                    detections.append(([x1, y1, x2-x1, y2-y1], conf, cls))

            # تتبع الأشخاص
            tracks = tracker.update_tracks(detections, frame=frame)

            current_ids = []

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                current_ids.append(track_id)

                # تحديث موقع الشخص في tracked_objects
                x1, y1, x2, y2 = track.to_ltrb()
                tracked_objects[track_id] = (x1, y1, x2, y2)

                # رسم المستطيل ومؤشر الطول في كل إطار
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)

                # حساب الطول الحقيقي للشخص
                box_height = y2 - y1
                person_height_cm = box_height * scale

                # عرض ID والطول فوق المستطيل
                cv2.putText(frame, f"ID: {track_id} Height: {int(person_height_cm)} cm",
                            (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

            # حفظ عدد الأشخاص الحالي
            people_count_per_frame.append(len(current_ids))

            # عرض عدد الأشخاص مباشرة على الفيديو
            cv2.putText(frame, f"People: {len(current_ids)}", (20,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)

            # حفظ الإطار في الفيديو الناتج
            out.write(frame)

            # عرض الفيديو أثناء المعالجة في Colab
            cv2_imshow(frame)

    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print(f"{video_path} processing done! Saved as {output_name}")

    # رسم مخطط عدد الأشخاص لكل إطار
    plt.figure(figsize=(12,5))
    plt.plot(people_count_per_frame, color='blue')
    plt.title(f"عدد الأشخاص في كل إطار - {video_path}", fontsize=14)
    plt.xlabel("Frame")
    plt.ylabel("عدد الأشخاص")
    plt.show()

# تحميل الفيديوهات الناتجة
for video_path in videos:
    files.download(f"output_{video_path}")


In [ ]:
# 1️⃣ تثبيت المكتبات
!pip install ultralytics opencv-python deep-sort-realtime mediapipe

# 2️⃣ استيراد المكتبات
import cv2
import mediapipe as mp
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import files

# 3️⃣ رفع الفيديوهات
uploaded = files.upload()
videos = list(uploaded.keys())

# 4️⃣ إعداد المتتبع
tracker = DeepSort(max_age=60, n_init=3)  # لضمان ثبات الـ ID لكل شخص

# 5️⃣ إعداد مكتبة Mediapipe للوجه
mp_face = mp.solutions.face_detection
face_detector = mp_face.FaceDetection(min_detection_confidence=0.5)

# 6️⃣ قاموس لتخزين الطول لكل شخص
person_heights = {}

# 7️⃣ معالجة كل فيديو
for video_path in videos:
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    output_name = f"output_{video_path}"
    out = cv2.VideoWriter(output_name, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detector.process(rgb_frame)

        detections = []
        if results.detections:
            for det in results.detections:
                bboxC = det.location_data.relative_bounding_box
                x1 = int(bboxC.xmin * width)
                y1 = int(bboxC.ymin * height)
                w_box = int(bboxC.width * width)
                h_box = int(bboxC.height * height)
                x2 = x1 + w_box
                y2 = y1 + h_box

                # الثقة والـ class
                conf = det.score[0] if det.score else 1.0
                cls = 0  # كل شخص class=0
                detections.append(([x1, y1, w_box, h_box], conf, cls))

        # تحديث DeepSort
        tracks = tracker.update_tracks(detections, frame=frame)

        for track in tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            x1, y1, x2, y2 = track.to_ltrb()

            # رسم المستطيل حول الوجه
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)

            # حساب الطول لأول مرة لكل شخص
            if track_id not in person_heights:
                face_height = y2 - y1
                person_height_cm = int(face_height * 7)  # تقدير تقريبي
                person_heights[track_id] = person_height_cm
            else:
                person_height_cm = person_heights[track_id]

            # عرض الـ ID والطول فوق رأس الشخص
            cv2.putText(frame, f"ID: {track_id} {person_height_cm} cm",
                        (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

        out.write(frame)
        cv2_imshow(frame)

    cap.release()
    out.release()
    print(f"{video_path} processing done! Saved as {output_name}")

# 8️⃣ تحميل الفيديوهات الناتجة
for video_path in videos:
    files.download(f"output_{video_path}")
